# Analysis of OpenStreetMap Data.

Pittsburgh, PA, United States

I have lived in Pittsburgh for the past 4 tears and I am interested in what the OSM will bring up.

# Following Code is From Project Details 

In [30]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "Pittsburgh.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

# The Code Below is to Find Out How Many Types of Tags There arte and the Number of Each Tag.



In [31]:

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re

filename = "Pittsburgh.osm"
# filename = "Sample.osm" #This is here so I can switch back and forth between the full .osm file and the sample

def count_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tags.keys():
            tags[element.tag] = 1
        else:
            tags[element.tag] += 1
    return tags

def test():

    tags = count_tags('Pittsburgh.osm')
    # tags = count_tags('Sample.osm') #This is here so I can switch back and forth between the full .osm file and the sample
    pprint.pprint(tags)

if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 69288,
 'meta': 1,
 'nd': 2252826,
 'node': 1860250,
 'note': 1,
 'osm': 1,
 'relation': 2765,
 'tag': 820435,
 'way': 323896}


# The Code Below Allows You to Check the k Value for Each Tag by Classifying the Tags into Few Categories:
1. "lower": valid tags containing only lowercase letters
2. "lower_colon": valid tags with a colon in the names
3. "problemchars": tags with problematic characters
4. "other": other tags that don't fall into the 3 categories above


In [32]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower,k):
            keys["lower"] += 1
        elif re.search(lower_colon,k):
            keys["lower_colon"] += 1
        elif re.search(problemchars,k):
            keys["problemchars"] += 1
        else:
            keys["other"] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

def test():
    keys = process_map('Pittsburgh.osm')
    # keys = process_map('Sample.osm') #This is here so I can switch back and forth between the full .osm file and the sample
    pprint.pprint(keys)

if __name__ == "__main__":
    test()

{'lower': 543151, 'lower_colon': 269335, 'other': 7947, 'problemchars': 2}


# Code Below Lists Street Types Not in the Expected List.

In [33]:
OSMFILE = "Pittsburgh.osm"
# OSMFILE = "sample.osm" #This is here so I can switch back and forth between the full .osm file and the sample
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Terrace", "Way"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

st_types = audit(OSMFILE)

def test():    
    pprint.pprint(dict(st_types))

if __name__ == '__main__':
    test()

{'22': set(['Business Route 22']),
 '48': set(['48']),
 'Alley': set(['Clay Alley',
               'Delmar Alley',
               'McGuinness Alley',
               'Oak Alley',
               'Park Alley',
               'Vine Alley']),
 'Allies': set(['Boulevard of the Allies']),
 'Automotive': set(['California Automotive']),
 'Ave': set(['1st Ave',
             '5th Ave',
             'Arlington Ave',
             'Centre Ave',
             'E Warrington Ave',
             'Elizabeth Ave',
             'Fifth Ave',
             'Forbes Ave',
             'Friendship Ave',
             'Greydon Ave',
             'Highland Ave',
             'Liberty Ave',
             'Lynnwood Ave',
             'Morewood Ave',
             'Penn Ave',
             'S Millvale Ave',
             'S Negley Ave',
             'S. Aiken Ave',
             'Shadeland Ave',
             'Shady Ave',
             'South Aiken Ave',
             'W Liberty Ave']),
 'Ave.': set(['4th Ave.', 'Chartiers Ave.

# Code updates the unexpected streets listed in the mapping list while keeping other streets unchanged.

In [34]:
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Rd": "Road",
            "Ter" : "Terrece",
            "Pl" : "Place",
           "Sq" : "Square"
            }

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m.group() not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    return name

def test():
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)

if __name__ == '__main__':
    test()

('Waterfront Drive West', '=>', 'Waterfront Drive West')
('Byrant St.', '=>', 'Byrant Street')
('Bryant St.', '=>', 'Bryant Street')
('West 11th St.', '=>', 'West 11th Street')
('Atwood St.', '=>', 'Atwood Street')
('Swindell Brdg', '=>', 'Swindell Brdg')
('Patricia', '=>', 'Patricia')
('Browns Hill Rd', '=>', 'Browns Hill Road')
('Jefferson Rd', '=>', 'Jefferson Road')
('Bayard Rd', '=>', 'Bayard Road')
('Library Rd', '=>', 'Library Road')
('520 Unity Center Rd', '=>', '520 Unity Center Road')
('Brownsville Rd', '=>', 'Brownsville Road')
('Duff Rd', '=>', 'Duff Road')
('McNeilly Rd', '=>', 'McNeilly Road')
('Boulevard of the Allies', '=>', 'Boulevard of the Allies')
('The Oaks', '=>', 'The Oaks')
('Waterfront Drive East', '=>', 'Waterfront Drive East')
('William Penn Highway', '=>', 'William Penn Highway')
('Perry Highway', '=>', 'Perry Highway')
('Lincoln Highway', '=>', 'Lincoln Highway')
('Tri-Boro Expressway', '=>', 'Tri-Boro Expressway')
('The Maples', '=>', 'The Maples')
('Busin

In [35]:
'''
This code checks for zipcode whether they begin with '94' or '95' or something else
'''
OSMFILE = "Pittsburgh.osm"
# OSMFILE = "sample.osm" #This is here so I can switch back and forth between the full .osm file and the sample
zip_type_re = re.compile(r'\d{5}$')

def audit_ziptype(zip_types, zipcode):
    if zipcode[0:2] != 15:
        zip_types[zipcode[0:2]].add(zipcode)
    elif zipcode[0:2] != 16:
        zip_types[zipcode[0:2]].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    zip_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_ziptype(zip_types,tag.attrib['v'])
    osm_file.close()
    return zip_types

zip_print = audit_zip(OSMFILE)

def test():    
    pprint.pprint(dict(zip_print))

if __name__ == '__main__':
    test()

{'15': set(['15017',
            '15031',
            '15034',
            '15035',
            '15057',
            '15064',
            '15102',
            '15104',
            '15106',
            '15108',
            '15110',
            '15112',
            '15120',
            '15120-5000',
            '15120-9998',
            '15122',
            '15132',
            '15136',
            '151363',
            '15137',
            '15142',
            '15145',
            '15146',
            '15147',
            '15148',
            '15201',
            '15202',
            '15203',
            '15203-2275',
            '15204',
            '15205',
            '15206',
            '15206-2011',
            '15206-2032',
            '15206-2033',
            '15206-2038',
            '15206-2058',
            '15206-2396',
            '15206-2696',
            '15206-2697',
            '15206-3807',
            '15206-4320',
            '15206-4336',
            '15206-4403',


# This Code Will Update Non 5-digit Zipcodes.
* If it is a 8 or 9 digit only the first 5 digits will be kept.
* If it has the state name in front, only the 5 digits are kept.
* If it is something else, will not change anything as it might result in error when validating the csv file.

In [36]:
def update_zipcode(zipcode):
    if re.findall(r'^\d{5}$', zipcode): # 5 digits 94140
        valid_zipcode = zipcode
        return valid_zipcode
    elif re.findall(r'(^\d{5})-\d{3}$', zipcode): # 8 digits 94150-029
        valid_zipcode = re.findall(r'(^\d{5})-\d{3}$', zipcode)[0]
        return valid_zipcode
    elif re.findall(r'(^\d{5})-\d{4}$', zipcode): # 9 digits 95130-0239
        valid_zipcode = re.findall(r'(^\d{5})-\d{4}$', zipcode)[0]
        return valid_zipcode
    elif re.findall(r'CA\s*\d{5}', zipcode): # with state code CA 95130
        valid_zipcode =re.findall(r'\d{5}', zipcode)[0]  
        return valid_zipcode  
    else: #return default zipcode to avoid overwriting
        return zipcode
    
def test_zip():
    for zips, ways in zip_print.iteritems():
        for name in ways:
            better_name = update_zipcode(name)
            print name, "=>", better_name

if __name__ == '__main__':
    test_zip()

15206-4320 => 15206
15205 => 15205
15204 => 15204
15207 => 15207
15206 => 15206
15201 => 15201
15203 => 15203
15202 => 15202
15209 => 15209
15208 => 15208
15213-1705 => 15213
15213-2608 => 15213
15206-4449 => 15206
15232-1845 => 15232
15213-1763 => 15213
15206-5311 => 15206
15213-3704 => 15213
15110 => 15110
15290 => 15290
15112 => 15112
15216 => 15216
15217 => 15217
15214 => 15214
15215 => 15215
15212 => 15212
15213 => 15213
15210 => 15210
15211 => 15211
15057 => 15057
15218 => 15218
15219 => 15219
15203-2275 => 15203
15213-1678 => 15213
15210-1845 => 15210
15206-4471 => 15206
15206-4472 => 15206
15232-2210 => 15232
15206-2038 => 15206
15213-2712 => 15213
15213-1503 => 15213
15206-3807 => 15206
15108 => 15108
15106 => 15106
15104 => 15104
15102 => 15102
15223 => 15223
15222 => 15222
15221 => 15221
15220 => 15220
15227 => 15227
15226 => 15226
15225 => 15225
15224 => 15224
15229 => 15229
15228 => 15228
15206-2032 => 15206
151363 => 151363
15232-1879 => 15232
15213-4026 => 15213
15232-14

# Following Code is from Udacity Lesson 13 Scema.py file.

In [37]:

import csv
import codecs
import cerberus
import schema

# %load schema.py

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

# Following Code is from Udacity Lesson 13 Data.py file.

In [38]:
OSM_PATH = "Pittsburgh.osm"
OSMFILE = "Pittsburgh.osm"

#OSM_PATH = "Sample.osm"  #This is here so I can switch back and forth between the full .osm file and the sample code
#OSMFILE = "Sample.osm"   #This is here so I can switch back and forth between the full .osm file and the sample code

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Terrace", "Way"]

mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "Rd": "Road"
            }

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']
    
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  
    p=0
    
    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            node_tags_attribs = {}
            temp = LOWER_COLON.search(tag.attrib['k'])
            is_p = PROBLEMCHARS.search(tag.attrib['k'])
            if is_p:
                continue
            elif temp:
                split_char = temp.group(1)
                split_index = tag.attrib['k'].index(split_char)
                type1 = temp.group(1)
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k'][split_index+2:]
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = tag.attrib['k'][:split_index+1]
                if node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "street":
                    node_tags_attribs['value'] = update_name(tag.attrib['v'], mapping) 
            else:
                node_tags_attribs['id'] = element.attrib['id']
                node_tags_attribs['key'] = tag.attrib['k']
                node_tags_attribs['value'] = tag.attrib['v']
                node_tags_attribs['type'] = 'regular'
                if node_tags_attribs['type'] == "addr" and node_tags_attribs['key'] == "street":
                    node_tags_attribs['value'] = update_name(tag.attrib['v'], mapping) 
            tags.append(node_tags_attribs)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        id = element.attrib['id']
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for i in element.iter('nd'):
            d = {}
            d['id'] = id
            d['node_id'] = i.attrib['ref']
            d['position'] = p
            p+=1
            way_nodes.append(d)
        for c in element.iter('tag'):
            temp = LOWER_COLON.search(c.attrib['k'])
            is_p = PROBLEMCHARS.search(c.attrib['k'])
            e = {}
            if is_p:
                continue
            elif temp:
                split_char = temp.group(1)
                split_index = c.attrib['k'].index(split_char)
                e['id'] = id
                e['key'] = c.attrib['k'][split_index+2:]
                e['type'] = c.attrib['k'][:split_index+1]
                e['value'] = c.attrib['v']
                if e['type'] == "addr" and e['key'] == "street":
                    e['value'] = update_name(c.attrib['v'], mapping) 
            else:
                e['id'] = id
                e['key'] = c.attrib['k']
                e['type'] = 'regular'
                e['value'] =  c.attrib['v']
                if e['type'] == "addr" and e['key'] == "street":
                    e['value'] = update_name(c.attrib['v'], mapping) 
            tags.append(e)
        
    return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
        codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
        codecs.open(WAYS_PATH, 'wb') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
        codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)

# Create DB and tables

In [39]:
import sqlite3
conn = sqlite3.connect('C750.sqlite')

In [53]:
conn.text_factory = str
cur = conn.cursor()

#Make new tables
cur.execute('''DROP TABLE IF EXISTS nodes''')
cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
cur.execute('''DROP TABLE IF EXISTS ways''')
cur.execute('''DROP TABLE IF EXISTS ways_tags''')
cur.execute('''DROP TABLE IF EXISTS ways_nodes''')


cur.execute('''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('nodes.csv','r') as nodes_table:
    dr = csv.DictReader(nodes_table)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO nodes VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)


cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')

with open('nodes_tags.csv','r') as nodes_tags_table:
    dr = csv.DictReader(nodes_tags_table)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags VALUES (?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')

with open('ways.csv','r') as ways_table:
    dr = csv.DictReader(ways_table)
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways VALUES (?, ?, ?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')

with open('ways_tags.csv','r') as ways_tags_table:
    dr = csv.DictReader(ways_tags_table)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags VALUES (?, ?, ?, ?);", to_db)

cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
''')

with open('ways_nodes.csv','r') as ways_nodes_table:
    dr = csv.DictReader(ways_nodes_table)
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes VALUES (?, ?, ?);", to_db)

#Save changes to DB
conn.commit()

# Show File size of Files

In [64]:

import os

print ('The Pittsburgh.osm file is {} MB'.format(os.path.getsize('Pittsburgh.osm')/1.0e6))
print ('The Sample.osm file is {} MB'.format(os.path.getsize('Sample.osm')/1.0e6))
print ('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print ('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes_tags.csv')/1.0e6))
print ('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print ('The ways_nodes.csv file is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6))
print ('The ways_tags.csv file is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print ('The C750.sqlite file is {} MB'.format(os.path.getsize('C750.sqlite')/1.0e6))

The Pittsburgh.osm file is 435.068972 MB
The Sample.osm file is 44.136175 MB
The nodes.csv file is 161.025194 MB
The nodes_tags.csv file is 3.648402 MB
The ways.csv file is 20.26941 MB
The ways_nodes.csv file is 54.190914 MB
The ways_tags.csv file is 24.593397 MB
The C750.sqlite file is 227.254272 MB


# Number of Ways

In [54]:
way_count = conn.cursor()
way_count.execute("SELECT COUNT(*) FROM ways")

way_count = way_count.fetchall()
 
for row in way_count:
    print(row)

(323896,)


# Number of Nodes

In [55]:
node_count = conn.cursor()
node_count.execute("SELECT COUNT(*) FROM nodes")

node_count = node_count.fetchall()
 
for row in node_count:
    print(row)

(1860250,)



# Unique Users

In [66]:
Total_Unique_users = conn.cursor()
Total_Unique_users.execute("SELECT COUNT(DISTINCT(e.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e")

Total_Unique_users = Total_Unique_users.fetchall()
 
for row in Total_Unique_users:
    print(row)

(1553,)


In [57]:

top_users = conn.cursor()
top_users.execute("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10")

top_users = top_users.fetchall()
 
for row in top_users:
    print(row)

('GeoKitten_import', 403814)
('Karthoo_import', 324888)
('Omnific', 249585)
('cowdog', 116924)
('DonovanG', 111671)
('thetornado76', 100752)
('abbafei', 83396)
('GeoKitten', 82133)
('Yaten_import', 67002)
('tmb926', 66594)


# Count of Amenities

In [65]:
amenities = conn.cursor()
t = ('amenity',)
amenities.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

amenities = amenities.fetchall()
 
for row in amenities:
    print(row)

('restaurant', 473)
('place_of_worship', 328)
('school', 257)
('bench', 208)
('waste_basket', 185)
('fast_food', 138)
('post_box', 135)
('parking_entrance', 129)
('library', 115)
('cafe', 101)


# Count of Tourism Amenities

In [59]:
tourism = conn.cursor()
t = ('tourism', 'tourism_1')
tourism.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value UNION SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC", t)

tourism = tourism.fetchall()
 
for row in tourism:
    print(row)

('artwork', 37)
('information', 22)
('hotel', 16)
('viewpoint', 16)
('museum', 9)
('picnic_site', 5)
('attraction', 3)
('gallery', 2)
('amusement_ride', 1)
('guest_house', 1)
('hostel', 1)
('motel', 1)


# Top Count of Cities

In [61]:
top_city = conn.cursor()
t = ('city',)
top_city.execute("SELECT value, COUNT(*) as num FROM nodes_tags WHERE key=? GROUP BY value ORDER BY num DESC LIMIT 10", t)

top_city = top_city.fetchall()
 
for row in top_city:
    print(row)

('Pittsburgh', 1487)
('North Versailles', 51)
('Penn Hills', 44)
('McKees Rocks', 36)
('Bridgeville', 26)
('Turtle Creek', 18)
('Carnegie', 14)
('Forest Hills', 13)
('Verona', 11)
('Aspinwall', 11)


# Analysis/Suggestions

When I was looking over the Postal codes I was surprised at how well the postal codes stuck to either 5 or 10 digit postal codes.  I was expecting to see multiple inconsistencies in how the postal codes were formatted.  This made for an easy standardization of the postal codes during clean up. It would be nice if there was a way to standardize street names as these fields are entered inconsistently.  However with OSM being an open source project I think this would cost a considerable amount of money in man hours or other resources to be able to standardize the inputs.

# Conclusion

This project was a great way to discover how people can join together to make data that can be used in a multitude of ways.  
While the data has its flaws it is able to be cleaned up and used in a variety of ways.  While it doesn't have the power behind such of google maps I can see OSM being a powerful tool for people looking for a free, open source area of information.